In [ ]:
#pip install geopandas
#pip install folium

In [ ]:
import os
import json
from urllib import request

import pandas as pd
import geopandas as gpd

import folium
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
# импорт функции rename_subj для приведения всех названий регионов к единому стандарту
path = os.getcwd()
os.chdir("../")
from Utils.subj_rename import rename_subj
os.chdir(path.split("\\")[-1])

In [ ]:
# импорт словаря переименований регионов для унифицирования их названия в используемых df
with open("../Utils/subj_rename.json", encoding="utf-8") as file:
    subj_rename_data = json.load(file)

In [ ]:
df_divorce = pd.read_excel("divorce.xls")
df_divorce["region"] = df_divorce["region"].str.strip()

# приведение названий регононов к единому стандарту для корректного отображения на карте
df_divorce = rename_subj(
    df_divorce,
    "region",
    subj_rename_data
)
display(df_divorce.head())
df_divorce.shape

In [ ]:
df_regions = gpd.read_file("../Utils/geo_data.json").rename(columns={"name": "region"})

# приведение названий регононов к единому стандарту для корректного отображения на карте
df_regions = rename_subj(
    df_regions,
    "region",
    subj_rename_data
)
display(df_regions.head())
df_regions.shape

In [ ]:
# Соединение датафреймов для отсеивания лишних полей по столбцу region
df_show = df_divorce.merge(df_regions, on="region", how="right")
display(df_show.head())
df_show.shape

In [ ]:
df_show[df_show.k.isna()]

# Отображение на карте

In [ ]:
russia_map = folium.Map(
    location = [64.6863136, 97.7453061],    # широта и долгота России
    zoom_start = 4
)

#display (russia_map)

In [ ]:
map = folium.Map([68 ,103],     
                        zoom_start=3,
                        width="%100",
                        height="%100")
folium.Choropleth(
    geo_data=df_regions,
    data=df_show,
    name = 'Разводы',
    columns=['region', 'k'],   
    legend_name='Коэффициент разводимости на 1000 человек',
    key_on='feature.properties.region',
    fill_color="PuBu",
    fill_opacity=0.8,
    line_opacity=0.5,
    ).add_to(map)
folium.LayerControl().add_to(map)

tooltip = "Click Here For More Info"
# координаты середины канала - 66°25' с. ш. 94°15' в. д
marker = folium.Marker(
    location=[66.25, 91.15],
    popup="<stong><a href='https://t.me/maps_stat'>tg: https://t.me/maps_stat</a><br /><a href='https://vk.com/maps_stat'>vk: https://vk.com/maps_stat</a></stong>",
    icon=folium.DivIcon(html=f"""
      <div style="color:#878787;background:#878787:90px;text-align:center;"><h5><u>@maps_stat</u><h5></div>
    """),
    fill_opacity=0.5
    )
marker.add_to(map)
map

In [ ]:
#map.save("divorce.html")

In [ ]:
# про Чукотку статья https://habr.com/ru/post/459902/